In [1]:
import os
import time
import datetime

from collections import Counter
import numpy as np
import pandas as pd
import unicodedata

from nltk.corpus import stopwords
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/Users/davidleifer/anaconda/lib/python3.5/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
data = '/Users/davidleifer/Downloads/bayes/merged.json'
df = pd.read_json(data, lines=True)

In [3]:
filename = '/Users/davidleifer/Downloads/bayes/merged_csv.csv'

df.to_csv(filename, index=False, encoding='utf-8')

In [4]:
tweets = pd.read_csv('/Users/davidleifer/Downloads/bayes/merged_csv.csv')

/Users/davidleifer/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0,1,2,3,4,5,6,9,10,13,19,20,21,23,28,29,30,31,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
tweets['geo'] = tweets['geo'].str.extract('.*\[(.*)\].*')

/Users/davidleifer/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


In [6]:
tweets['y'], tweets['x'] = tweets['geo'].str.split(',', 1).str

In [7]:
df = tweets

In [8]:
df = df[df['text'].notnull()]

In [9]:
stop = stopwords.words('english')
df['tweet_without_stopwords'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

/Users/davidleifer/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [10]:
stop =  ['The','RT','&amp;', '-', 'A', 'https:', '.', '2']
df['tweet_without_stopwords'] = df['tweet_without_stopwords'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

/Users/davidleifer/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [11]:
#remove periods
df['tweet_without_stopwords'] = df['tweet_without_stopwords'].str.replace('[\.]','')
#remove commas
df['tweet_without_stopwords'] = df['tweet_without_stopwords'].str.replace('[\,]','')
#remove -
df['tweet_without_stopwords'] = df['tweet_without_stopwords'].str.replace('[-]','')
#remove @
df['tweet_without_stopwords'] = df['tweet_without_stopwords'].str.replace('[@]','')

/Users/davidleifer/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/davidleifer/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/davidleifer/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

In [12]:
df.head()

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,retweeted_status,source,text,timestamp_ms,truncated,user,withheld_in_countries,y,x,tweet_without_stopwords
0,NaN,NaN,2017-08-02 02:53:46.000,NaN,"{'hashtags': [{'indices': [26, 35], 'text': 'B...",NaN,NaN,0.0,0.0,low,...,{'created_at': 'Wed Aug 02 00:10:02 +0000 2017...,"<a href=""http://twitter.com/download/android"" ...",RT @Beluga_Solar: Keep ur #Business costs down...,2017-08-02 02:53:46.798,0.0,{'created_at': 'Fri Jul 28 16:49:42 +0000 2017...,NaN,NaN,NaN,Beluga_Solar: Keep ur #Business costs installi...
1,NaN,NaN,2017-08-02 02:53:48.000,NaN,"{'hashtags': [{'indices': [70, 76], 'text': 's...",NaN,NaN,0.0,0.0,low,...,{'created_at': 'Wed Aug 02 02:00:07 +0000 2017...,"<a href=""http://twitter.com/download/android"" ...",RT @Beluga_Solar: We have a selection of Proje...,2017-08-02 02:53:48.072,0.0,{'created_at': 'Wed Feb 22 07:02:34 +0000 2017...,NaN,NaN,NaN,Beluga_Solar: We selection Project Lamps ready...
2,NaN,NaN,2017-08-02 02:53:50.000,NaN,"{'hashtags': [{'indices': [70, 76], 'text': 's...",NaN,NaN,0.0,0.0,low,...,{'created_at': 'Wed Aug 02 02:00:07 +0000 2017...,"<a href=""http://twitter.com/download/android"" ...",RT @Beluga_Solar: We have a selection of Proje...,2017-08-02 02:53:50.492,0.0,{'created_at': 'Sat Jun 17 09:06:20 +0000 2017...,NaN,NaN,NaN,Beluga_Solar: We selection Project Lamps ready...
3,NaN,NaN,2017-08-02 02:53:50.000,NaN,"{'hashtags': [{'indices': [18, 24], 'text': 'S...",NaN,NaN,0.0,0.0,low,...,{'created_at': 'Wed Aug 02 02:00:22 +0000 2017...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...","RT @Beluga_Solar: #Solar Speed Checking Light,...",2017-08-02 02:53:50.867,0.0,{'created_at': 'Wed Aug 02 02:20:21 +0000 2017...,NaN,NaN,NaN,Beluga_Solar: #Solar Speed Checking Light Wide...
4,NaN,NaN,2017-08-02 02:53:51.000,NaN,"{'hashtags': [{'indices': [70, 76], 'text': 's...",NaN,NaN,0.0,0.0,low,...,{'created_at': 'Wed Aug 02 02:00:07 +0000 2017...,"<a href=""http://twitter.com/download/android"" ...",RT @Beluga_Solar: We have a selection of Proje...,2017-08-02 02:53:51.142,0.0,{'created_at': 'Fri Jul 28 17:21:49 +0000 2017...,NaN,NaN,NaN,Beluga_Solar: We selection Project Lamps ready...


In [13]:
df["compound"] = ''
df["neg"] = ''
df["neu"] = ''
df["pos"] = ''
sid = SentimentIntensityAnalyzer()
    
for user, row in df.T.iteritems():
    try:
        sentence = unicodedata.normalize('NFKD', df.loc[user, 'tweet_without_stopwords'])
        ss = sid.polarity_scores(sentence)
        df.set_value(user, 'compound', ss['compound'])
        df.set_value(user, 'neg', ss['neg'])
        df.set_value(user, 'neu', ss['neu'])
        df.set_value(user, 'pos', ss['pos'])
    except TypeError:
        print(df.loc[user, 'tweet_without_stopwords'])

/Users/davidleifer/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/davidleifer/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/davidleifer/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [14]:
filename = '/Users/davidleifer/Downloads/bayes/merged_geo_csv.csv'

df.to_csv(filename, index=False, encoding='utf-8')